#### Create Train and Test Folders

In [2]:
!ls /home/etutubalina/BioSyn/datasets/

BC2GN			bc5cdr-disease.tar.gz  tac2017adr
MCN-disease		dictionary	       trials-cancer-intervention
bc5cdr-chemical		ncbi-disease	       trials-disease
bc5cdr-chemical.tar.gz	ncbi-disease.tar.gz    trials-intervention
bc5cdr-disease		raw


In [15]:
def process(file_ent, file_labels, save_to):
    with open(file_ent, encoding='utf-8') as input_stream:
        entities = input_stream.readlines()
    with open(file_labels, encoding='utf-8') as input_stream:
        labels = input_stream.readlines()
    with open(save_to+"0.concept","w+") as fout:
        for concept,cui in zip(entities,labels):
            concept = concept.strip()
            cui = cui.strip()
            fout.write('0||0|{}||Entity||{}||{}\n'.format(len(concept),concept,cui))

In [16]:
process("processed/train_med_entities.txt",
        "processed/train_labels.txt",
        "/home/etutubalina/BioSyn/datasets/MCN/train/")

In [17]:
process("processed/test_med_entities.txt",
        "processed/test_labels.txt",
        "/home/etutubalina/BioSyn/datasets/MCN/test/")

In [ ]:
/home/etutubalina/BioSyn/datasets/ncbi-disease/processed_test_wtht_train_dupl/
DATA/medical_processing_corpora/MCN_n2c2/processed/test_labels.txt

#### Create Test Wtht Duplicates

In [22]:
import glob
from tqdm import tqdm
from itertools import chain
files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_test/*.concept")
data = []
for concept_file in tqdm(files):
    with open(concept_file, "r", encoding='utf-8') as f:
        concepts = f.readlines()
    for concept in concepts:
        c = concept.split("||")
        mention = c[3].strip()
        cui = c[4].strip()
        data.append((mention,cui))
print(len(data))  
data = list(dict.fromkeys(data))
print(len(data))  
#print(data)
# with open("/home/etutubalina/BioSyn/datasets/ncbi-disease/processed_test_wtht_dupl/0.concept","w+") as fout:
#     for concept,cui in tqdm(data):
#         fout.write('0||0|{}||Disease||{}||{}\n'.format(len(concept),concept,cui))

                    
files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_train/*.concept")
train_data = []
for concept_file in tqdm(files):
    with open(concept_file, "r", encoding='utf-8') as f:
        concepts = f.readlines()
    for concept in concepts:
        c = concept.split("||")
        mention = c[3].strip()
        cui = c[4].strip()
        train_data.append((mention,cui))
print(len(train_data))  
train_data = list(dict.fromkeys(train_data))
print(len(train_data))  
data = list([x for x,y in dict.fromkeys(data).items() if x not in dict.fromkeys(train_data)])
print(len(data))  

test_ids = [x for x in set(chain(*[y.replace("+","|").split('|') for x,y in data]))]
train_ids = [x for x in set(chain(*[y.replace("+","|").split('|') for x,y in train_data]))]

print(len(set(set(test_ids).union(set(train_ids)))))
#print(set(set(test_ids).union(set(train_ids))))
with open("/home/etutubalina/BioSyn/datasets/MCN/processed_test_wtht_train_dupl/snomed.concept","w+") as fout:
    for concept,cui in tqdm(data):
        fout.write('0||0|{}||Entity||{}||{}\n'.format(len(concept),concept,cui))
with open("/home/etutubalina/BioSyn/datasets/MCN/processed_test_wtht_train_dupl/rxnorm.concept","w+") as fout:
    for concept,cui in tqdm(data):
        fout.write('0||0|{}||Entity||{}||{}\n'.format(len(concept),concept,cui))

100%|██████████| 50/50 [00:00<00:00, 5234.90it/s]
0it [00:00, ?it/s]
100%|██████████| 3783/3783 [00:00<00:00, 565347.82it/s]

6925
3783
0
0
3783
2579


#### Create Vocab for BioSyn

In [23]:
import pandas as pd

In [26]:
df_rxnorm=pd.read_csv("/home/etutubalina/DATA/UMLS/UMLS2019AB/UMLS_RXNORM.txt",names=['CUI','t'],sep='\t')

In [27]:
df_rxnorm.head()

,CUI,t
0,C0000039,"1,2-dipalmitoylphosphatidylcholine"
1,C0000266,Parlodel
2,C0000294,Mesna
3,C0000378,Droxidopa
4,C0000392,beta-Alanine


In [28]:
df_snm=pd.read_csv("/home/etutubalina/DATA/UMLS/UMLS2019AB/UMLS_SNOMEDCT_US.txt",names=['CUI','t'],sep='\t')

In [29]:
df_snm.head()

,CUI,t
0,C0000039,Dipalmitoylphosphatidylcholine
1,C0000039,Dipalmitoylphosphatidylcholine (substance)
2,C0000052,"1,4-alpha-Glucan branching enzyme"
3,C0000052,Branching enzyme
4,C0000052,"Amylo-(1,4,6)-transglycosylase"


In [31]:
[x for x in test_ids if x not in df_rxnorm.CUI.tolist() and x not in df_snm.CUI.tolist()]

['C3232296', 'C0117857', 'CUI-less', 'C0553982', 'C1827170']

In [42]:
snm_test_id = [x for x in test_ids if x in df_snm.CUI.tolist() and x not in df_rxnorm.CUI.tolist()]

In [43]:
rxnorm_test_id = [x for x in test_ids if x in df_rxnorm.CUI.tolist()] 

In [44]:
print(len(set(set(snm_test_id).intersection(set(rxnorm_test_id)))))

0


In [ ]:
set(set(snm_test_id).intersection(set(rxnorm_test_id)))

In [32]:
[x for x in train_ids if x not in df_rxnorm.CUI.tolist() and x not in df_snm.CUI.tolist()]

[]

In [33]:
df_all = pd.concat([df_rxnorm,df_snm])

In [36]:
def create_dictionary(df_current,save_to):
    with open(save_to,"w+") as fout:
    for e,group in df_current.groupby("t"):
        ids = '|'.join(group.CUI.unique().tolist())
        name = e.lower()
        fout.write(f'{ids}||{name}\n')

save_to='/home/etutubalina/BioSyn/datasets/MCN/test_dictionary.txt'
create_dictionary(save_to,df_all)

save_to='/home/etutubalina/BioSyn/datasets/MCN-disease/test_dictionary.txt'
create_dictionary(save_to,df_snmnm)

save_to='/home/etutubalina/BioSyn/datasets/MCN-chemical/test_dictionary.txt'
create_dictionary(save_to,df_rxnorm)

In [45]:
save_to='/home/etutubalina/BioSyn/datasets/MCN-disease/test_dictionary.txt'
with open(save_to,"w+") as fout:
    for e,group in df_all.groupby("t"):
        ids = '|'.join(group.CUI.unique().tolist())
        name = e.lower()
        fout.write(f'{ids}||{name}\n')

In [46]:
save_to='/home/etutubalina/BioSyn/datasets/MCN-chemical/test_dictionary.txt'
with open(save_to,"w+") as fout:
    for e,group in df_rxnorm.groupby("t"):
        ids = '|'.join(group.CUI.unique().tolist())
        name = e.lower()
        fout.write(f'{ids}||{name}\n')

In [50]:
import os
def filter_set(path_to,save_to, list_in=[], list_out=[]):
    train_data = []
    list_in = set(list_in)
    list_out = set(list_out)
    for concept_file in tqdm(files):
        with open(concept_file, "r", encoding='utf-8') as f:
            concepts = f.readlines()
        with open(save_to+os.path.basename(concept_file), "w", encoding='utf-8') as f:    
            for concept in concepts:
                c = concept.split("||")
                cui = c[4].strip()
                if cui in list_in and cui not in list_out:
                    f.write(concept)

files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_train/*.concept")
save_to = "/home/etutubalina/BioSyn/datasets/MCN-disease/processed_train/"
filter_set(files,save_to,df_snm.CUI.tolist(),df_rxnorm.CUI.tolist())

files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_test_wtht_train_dupl/*.concept")
save_to = "/home/etutubalina/BioSyn/datasets/MCN-disease/processed_test_wtht_train_dupl/"
filter_set(files,save_to,df_snm.CUI.tolist(),df_rxnorm.CUI.tolist())

files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_train/*.concept")
save_to = "/home/etutubalina/BioSyn/datasets/MCN-chemical/processed_train/"
filter_set(files,save_to,df_rxnorm.CUI.tolist(),[])

files = glob.glob("/home/etutubalina/BioSyn/datasets/MCN/processed_test_wtht_train_dupl/*.concept")
save_to = "/home/etutubalina/BioSyn/datasets/MCN-chemical/processed_test_wtht_train_dupl/"
filter_set(files,save_to,df_rxnorm.CUI.tolist(),[])

100%|██████████| 1/1 [00:00<00:00, 238.38it/s]
